##### 1. Setup working directory and path to study area

In [1]:
%cd /home/WUR/duku002/Scripts/drought_floods/vscode


/home/WUR/duku002/Scripts/drought_floods/vscode


In [2]:
working_dir='/lustre/backup/WUR/ESG/duku002/Drought-Flood-Cascade/parana'
study_area='/home/WUR/duku002/Scripts/NBAT/hydro/common_data/parana.shp'

##### 2 .Download and preprocess input data

In [ ]:

# download and preprocess MODIS vegetation continuous fields from Google Earth Engine Data catalog

from bakaano.tree_cover import TreeCover
vf = TreeCover(
    working_dir=working_dir, 
    study_area=study_area, 
    start_date='2001-01-01', 
    end_date='2020-12-31'
)
vf.get_tree_cover_data()
vf.plot_tree_cover(variable='tree_cover') # options for plot are 'tree_cover' and 'herb_cover'

In [ ]:
# download and preprocess MODIS NDVI data from Google Earth Engine Data catalog

from bakaano.ndvi import NDVI
nd = NDVI(
    working_dir=working_dir, 
    study_area=study_area, 
    start_date='2001-01-01', 
    end_date='2010-12-31'
)
nd.get_ndvi_data()
nd.plot_ndvi(interval_num=10)  # because NDVI is in 16-day interval the 'interval_num' represents a 16-day period. 
                               #Hence 0 is the first 16 day period

In [ ]:
# Get elevation data

from bakaano.dem import DEM
dd = DEM(
    working_dir=working_dir, 
    study_area=study_area, 
    local_data=False, 
    local_data_path=None
)
dd.get_dem_data()
dd.plot_dem()

In [ ]:
# Get soil data

from bakaano.soil import Soil
sgd = Soil(
    working_dir=working_dir, 
    study_area=study_area
)
sgd.get_soil_data()
sgd.plot_soil(variable='wilting_point')  #options are 'wilting_point', 'saturation_point' and 'available_water_content'

In [ ]:
#  Get alpha earth satellite embedding dataset

from bakaano.alpha_earth import AlphaEarth
dd = AlphaEarth(
    working_dir=working_dir, 
    study_area=study_area,
    start_date='2013-01-01', 
    end_date = '2024-01-01'
)
dd.get_alpha_earth()
dd.plot_alpha_earth('A35') #Band options are A00 to A63

In [ ]:
#get meteo

from bakaano.meteo import Meteo
cd = Meteo(
    working_dir=working_dir, 
    study_area=study_area, 
    start_date='2001-01-01', 
    end_date='2010-12-31',
    local_data=False, 
    data_source='ERA5'
)
cd.plot_meteo(variable='tasmin', date='2006-12-01') # variable options are 'tmean', 'precip', 'tasmax', 'tasmin'

#####   3. Computing runoff and routing to river network

In [3]:

from bakaano.veget import VegET
vg = VegET(
    working_dir=working_dir, 
    study_area=study_area,
    start_date='2001-01-01', 
    end_date='2010-12-31',
    climate_data_source='ERA5',
    routing_method='mfd'
)
vg.compute_veget_runoff_route_flow()

Routed runoff data exists in /lustre/backup/WUR/ESG/duku002/Drought-Flood-Cascade/parana/runoff_output/wacc_sparse_arrays.pkl. Skipping processing


In [4]:
#visualize routed runoff data

from bakaano.plot_runoff import RoutedRunoff
rr = RoutedRunoff(
    working_dir=working_dir, 
    study_area=study_area
)
#rr.map_routed_runoff(date='2020-01-03', vmax=7) #output values have been log transformed for better visualization

In [ ]:
rr.interactive_plot_routed_runoff_timeseries(start_date='2000-01-01', end_date='2000-12-31', 
                                             grdc_netcdf='/lustre/backup/WUR/ESG/duku002/NBAT/hydro/input_data/GRDC-Daily-africa-south-america.nc'
                                 )

Available station_ids:
3264500, 3265005, 3265010, 3265050, 3265070, 3265100, 3265300, 3265600, 3265601, 3265700, 3265810, 3265815, 3265910, 3268100, 3268210, 3268270, 3268310, 3268500, 3268900, 3368100, 3368110, 3659100, 3659105, 3660200, 3660201, 3660600, 3660602, 3660700, 3660730, 3660900, 3661300, 3661600, 3661700, 3662100, 3662150, 3662300, 3662400, 3662500, 3662550, 3662600, 3662650, 3662655, 3662700, 3663700, 3663800, 3664100, 3664102, 3664150, 3664160, 3664600, 3664800, 3664802, 3664804, 3664905, 3666020, 3666050, 3666100, 3666200, 3666250, 3666300, 3666400, 3666401, 3666500, 3666505, 3666600, 3666700, 3667050, 3667060, 3667400


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x148b48532890>>
Traceback (most recent call last):
  File "/home/WUR/duku002/miniconda3/envs/bakaano-hydro/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


#####   4. Explore input data, river networks and hydrological stations interactively

In [ ]:
from bakaano.runner import BakaanoHydro
bk = BakaanoHydro(
    working_dir=working_dir, 
    study_area=study_area,
    climate_data_source='ERA5'
)
bk.explore_data_interactively('1981-01-01', '2016-12-31', '/lustre/backup/WUR/ESG/duku002/NBAT/hydro/input_data/GRDC-Daily-africa-south-america.nc')

#####   5. Training, Evaluating and Applying Bakaano-Hydro model 

In [ ]:
# INITIALIZE INSTANCE OF BAKAANO-HYDRO MODEL

from bakaano.runner import BakaanoHydro
bk = BakaanoHydro(  
    working_dir=working_dir, 
    study_area=study_area,
    climate_data_source='ERA5'
)

In [ ]:
# TRAINING BAKAANO-HYDRO MODEL

# The model is trained using the GRDC streamflow data.
# Note: The training process is computationally expensive and may take a long time to complete.
# trained model is always in the models folder in the working_dir and with a .keras extension


bk.train_streamflow_model(
    train_start='1981-01-01', 
    train_end='2020-12-31', 
    grdc_netcdf='/lustre/backup/WUR/ESG/duku002/NBAT/hydro/input_data/GRDC-Daily-africa-south-america.nc', 
    batch_size=32, 
    num_epochs=300,
    learning_rate=0.001  #for fewer stations reduce learning rate to 0.0001
)

#####   5c. Optional: Evaluate/Simulate with observed CSV files
Use the same lookup table and station CSV directory as training.


In [ ]:
# Evaluate interactively using CSV observations (enter station id when prompted)
model_path = f'{working_dir}/models/bakaano_model.keras'

bk.evaluate_streamflow_model_interactively(
    model_path=model_path,
    val_start='2001-01-01',
    val_end='2010-12-31',
    grdc_netcdf=None,
    csv_dir='/path/to/observed_csvs',
    lookup_csv='/path/to/station_lookup.csv'
)

# Batch prediction for stations listed in lookup CSV
bk.simulate_grdc_stations(
    model_path=model_path,
    sim_start='1981-01-01',
    sim_end='2020-12-31',
    grdc_netcdf=None,
    csv_dir='/path/to/observed_csvs',
    lookup_csv='/path/to/station_lookup.csv'
)


In [ ]:
# EVALUATING THE TRAINED MODEL INTERACTIVELY

# The model is evaluated using the GRDC streamflow data.


# trained model is always in the models folder in the working_dir and with a .keras extension
# the model names is always in the format: bakaano_model_<loss_fn>_<num_input_branch>_branches.keras
model_path = f'{working_dir}/models/bakaano_model.keras' 

bk.evaluate_streamflow_model_interactively(
    model_path=model_path, 
    val_start='2001-01-01', 
    val_end='2010-12-31', 
    grdc_netcdf='/lustre/backup/WUR/ESG/duku002/NBAT/hydro/input_data/GRDC-Daily-africa-south-america.nc'
)

In [ ]:
# Batch prediction for GRDC stations

model_path = f'{working_dir}/models/bakaano_model.keras'

bk.simulate_grdc_stations(
    model_path=model_path, 
    sim_start='1981-01-01', 
    sim_end='2020-12-31', 
    grdc_netcdf='/lustre/backup/WUR/ESG/duku002/NBAT/hydro/input_data/GRDC-Daily-africa-south-america.nc'
)

In [ ]:
# PREDICTING STREAMFLOW USING THE TRAINED MODEL AND STORING AS CSV FILES 
# The model is used to predict streamflow in any location in the study area. 

model_path = f'{working_dir}/models/bakaano_model.keras'

bk.simulate_streamflow(
    model_path=model_path, 
    sim_start='1981-01-01', 
    sim_end='1990-12-31', 
    latlist=[13.8, 13.9],
    lonlist=[3.0, 4.0]
)
